<a href="https://colab.research.google.com/github/Rishabh-a-git/D606-Thyroid_Classification/blob/main/CAPSTONE_PROJECT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [294]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Import Dataset

In [295]:
df=pd.read_csv('thyroidDF.csv')

In [296]:
df

,age,sex,on_thyroxine,query_on_thyroxine,on_antithyroid_meds,sick,pregnant,thyroid_surgery,I131_treatment,query_hypothyroid,...,TT4,T4U_measured,T4U,FTI_measured,FTI,TBG_measured,TBG,referral_source,target,patient_id
0,29,F,f,f,f,f,f,f,f,t,...,NaN,f,NaN,f,NaN,f,NaN,other,-,840801013
1,29,F,f,f,f,f,f,f,f,f,...,128.0,f,NaN,f,NaN,f,NaN,other,-,840801014
2,41,F,f,f,f,f,f,f,f,f,...,NaN,f,NaN,f,NaN,t,11.0,other,-,840801042
3,36,F,f,f,f,f,f,f,f,f,...,NaN,f,NaN,f,NaN,t,26.0,other,-,840803046
4,32,F,f,f,f,f,f,f,f,f,...,NaN,f,NaN,f,NaN,t,36.0,other,S,840803047
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9167,56,M,f,f,f,f,f,f,f,f,...,64.0,t,0.83,t,77.0,f,NaN,SVI,-,870119022
9168,22,M,f,f,f,f,f,f,f,f,...,91.0,t,0.92,t,99.0,f,NaN,SVI,-,870119023
9169,69,M,f,f,f,f,f,f,f,f,...,113.0,t,1.27,t,89.0,f,NaN,SVI,I,870119025
9170,47,F,f,f,f,f,f,f,f,f,...,75.0,t,0.85,t,88.0,f,NaN,other,-,870119027


# Data Preprocessing

In [297]:
df.dtypes

age                      int64
sex                     object
on_thyroxine            object
query_on_thyroxine      object
on_antithyroid_meds     object
sick                    object
pregnant                object
thyroid_surgery         object
I131_treatment          object
query_hypothyroid       object
query_hyperthyroid      object
lithium                 object
goitre                  object
tumor                   object
hypopituitary           object
psych                   object
TSH_measured            object
TSH                    float64
T3_measured             object
T3                     float64
TT4_measured            object
TT4                    float64
T4U_measured            object
T4U                    float64
FTI_measured            object
FTI                    float64
TBG_measured            object
TBG                    float64
referral_source         object
target                  object
patient_id               int64
dtype: object

### Drop unnecessary columns

In [298]:

df=df.drop(columns=['TSH_measured','T3_measured','TT4_measured','T4U_measured','FTI_measured','TBG_measured','TBG'],axis=1)

In [299]:
df.describe()

,age,TSH,T3,TT4,T4U,FTI,patient_id
count,9172.000000,8330.000000,6568.000000,8730.000000,8363.000000,8370.000000,9.172000e+03
mean,73.555822,5.218403,1.970629,108.700305,0.976056,113.640746,8.529473e+08
std,1183.976718,24.184006,0.887579,37.522670,0.200360,41.551650,7.581969e+06
min,1.000000,0.005000,0.050000,2.000000,0.170000,1.400000,8.408010e+08
25%,37.000000,0.460000,1.500000,87.000000,0.860000,93.000000,8.504090e+08
50%,55.000000,1.400000,1.900000,104.000000,0.960000,109.000000,8.510040e+08
75%,68.000000,2.700000,2.300000,126.000000,1.065000,128.000000,8.607110e+08
max,65526.000000,530.000000,18.000000,600.000000,2.330000,881.000000,8.701190e+08


In [300]:
df.isnull().sum()

age                       0
sex                     307
on_thyroxine              0
query_on_thyroxine        0
on_antithyroid_meds       0
sick                      0
pregnant                  0
thyroid_surgery           0
I131_treatment            0
query_hypothyroid         0
query_hyperthyroid        0
lithium                   0
goitre                    0
tumor                     0
hypopituitary             0
psych                     0
TSH                     842
T3                     2604
TT4                     442
T4U                     809
FTI                     802
referral_source           0
target                    0
patient_id                0
dtype: int64

## Looking at NAN values and age range


In [301]:
age_column = df['age']

# Calculate the IQR for the 'age' column
Q1_age = age_column.quantile(0.25)
Q3_age = age_column.quantile(0.75)
IQR_age = Q3_age - Q1_age

# Define a threshold for identifying outliers (e.g., 1.5 times the IQR)
outlier_threshold_age = 1.5

# Find outliers in the 'age' column
outliers_age = (age_column < (Q1_age - outlier_threshold_age * IQR_age)) | (age_column > (Q3_age + outlier_threshold_age * IQR_age))

# Display the rows containing outliers in the 'age' column
outliers_rows_age = df[outliers_age]
print("Rows containing outliers in the 'age' column:")
print(outliers_rows_age)

Rows containing outliers in the 'age' column:
        age sex on_thyroxine query_on_thyroxine on_antithyroid_meds sick  \
2976    455   F            f                  f                   f    f   
5710  65511   M            f                  f                   f    f   
6392  65512   M            f                  f                   f    f   
8105  65526   F            f                  f                   f    f   

     pregnant thyroid_surgery I131_treatment query_hypothyroid  ...  \
2976        f               f              f                 f  ...   
5710        f               f              f                 f  ...   
6392        f               f              f                 f  ...   
8105        f               f              f                 f  ...   

     hypopituitary psych   TSH   T3    TT4   T4U    FTI  referral_source  \
2976             f     f  1.10  2.0  118.0  1.13  104.0              SVI   
5710             f     f  0.38  2.0  113.0  1.08  104.0          

In [302]:
df = df[df['age'] <= 100]

# Display the dataset after filtering
print("Dataset after removing rows with 'age' above 100:")
df

Dataset after removing rows with 'age' above 100:


,age,sex,on_thyroxine,query_on_thyroxine,on_antithyroid_meds,sick,pregnant,thyroid_surgery,I131_treatment,query_hypothyroid,...,hypopituitary,psych,TSH,T3,TT4,T4U,FTI,referral_source,target,patient_id
0,29,F,f,f,f,f,f,f,f,t,...,f,f,0.3,NaN,NaN,NaN,NaN,other,-,840801013
1,29,F,f,f,f,f,f,f,f,f,...,f,f,1.6,1.9,128.0,NaN,NaN,other,-,840801014
2,41,F,f,f,f,f,f,f,f,f,...,f,f,NaN,NaN,NaN,NaN,NaN,other,-,840801042
3,36,F,f,f,f,f,f,f,f,f,...,f,f,NaN,NaN,NaN,NaN,NaN,other,-,840803046
4,32,F,f,f,f,f,f,f,f,f,...,f,f,NaN,NaN,NaN,NaN,NaN,other,S,840803047
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9167,56,M,f,f,f,f,f,f,f,f,...,f,f,NaN,NaN,64.0,0.83,77.0,SVI,-,870119022
9168,22,M,f,f,f,f,f,f,f,f,...,f,f,NaN,NaN,91.0,0.92,99.0,SVI,-,870119023
9169,69,M,f,f,f,f,f,f,f,f,...,f,f,NaN,NaN,113.0,1.27,89.0,SVI,I,870119025
9170,47,F,f,f,f,f,f,f,f,f,...,f,f,NaN,NaN,75.0,0.85,88.0,other,-,870119027


In [319]:
# Null values in sex column
null_sex_rows = df[df['sex'].isnull()]
null_sex_rows

,age,sex,on_thyroxine,query_on_thyroxine,on_antithyroid_meds,sick,pregnant,thyroid_surgery,I131_treatment,query_hypothyroid,...,hypopituitary,psych,TSH,T3,TT4,T4U,FTI,referral_source,class,Patient_ID


In [305]:
target_counts = null_sex_rows['target'].value_counts()
target_counts

-      212
I       17
G       16
R       13
K       11
D        7
F        7
A        6
J        3
N        3
B        2
M        2
S        2
KJ       1
L        1
AK       1
C|I      1
GK       1
FK       1
Name: target, dtype: int64

In [306]:
df = df.dropna(subset=['sex'])
df

,age,sex,on_thyroxine,query_on_thyroxine,on_antithyroid_meds,sick,pregnant,thyroid_surgery,I131_treatment,query_hypothyroid,...,hypopituitary,psych,TSH,T3,TT4,T4U,FTI,referral_source,target,patient_id
0,29,F,f,f,f,f,f,f,f,t,...,f,f,0.3,NaN,NaN,NaN,NaN,other,-,840801013
1,29,F,f,f,f,f,f,f,f,f,...,f,f,1.6,1.9,128.0,NaN,NaN,other,-,840801014
2,41,F,f,f,f,f,f,f,f,f,...,f,f,NaN,NaN,NaN,NaN,NaN,other,-,840801042
3,36,F,f,f,f,f,f,f,f,f,...,f,f,NaN,NaN,NaN,NaN,NaN,other,-,840803046
4,32,F,f,f,f,f,f,f,f,f,...,f,f,NaN,NaN,NaN,NaN,NaN,other,S,840803047
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9167,56,M,f,f,f,f,f,f,f,f,...,f,f,NaN,NaN,64.0,0.83,77.0,SVI,-,870119022
9168,22,M,f,f,f,f,f,f,f,f,...,f,f,NaN,NaN,91.0,0.92,99.0,SVI,-,870119023
9169,69,M,f,f,f,f,f,f,f,f,...,f,f,NaN,NaN,113.0,1.27,89.0,SVI,I,870119025
9170,47,F,f,f,f,f,f,f,f,f,...,f,f,NaN,NaN,75.0,0.85,88.0,other,-,870119027


In [307]:
df.reset_index(drop=True, inplace=True)
df

,age,sex,on_thyroxine,query_on_thyroxine,on_antithyroid_meds,sick,pregnant,thyroid_surgery,I131_treatment,query_hypothyroid,...,hypopituitary,psych,TSH,T3,TT4,T4U,FTI,referral_source,target,patient_id
0,29,F,f,f,f,f,f,f,f,t,...,f,f,0.3,NaN,NaN,NaN,NaN,other,-,840801013
1,29,F,f,f,f,f,f,f,f,f,...,f,f,1.6,1.9,128.0,NaN,NaN,other,-,840801014
2,41,F,f,f,f,f,f,f,f,f,...,f,f,NaN,NaN,NaN,NaN,NaN,other,-,840801042
3,36,F,f,f,f,f,f,f,f,f,...,f,f,NaN,NaN,NaN,NaN,NaN,other,-,840803046
4,32,F,f,f,f,f,f,f,f,f,...,f,f,NaN,NaN,NaN,NaN,NaN,other,S,840803047
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8856,56,M,f,f,f,f,f,f,f,f,...,f,f,NaN,NaN,64.0,0.83,77.0,SVI,-,870119022
8857,22,M,f,f,f,f,f,f,f,f,...,f,f,NaN,NaN,91.0,0.92,99.0,SVI,-,870119023
8858,69,M,f,f,f,f,f,f,f,f,...,f,f,NaN,NaN,113.0,1.27,89.0,SVI,I,870119025
8859,47,F,f,f,f,f,f,f,f,f,...,f,f,NaN,NaN,75.0,0.85,88.0,other,-,870119027


In [308]:
df.describe()

,age,TSH,T3,TT4,T4U,FTI,patient_id
count,8861.000000,8048.000000,6341.000000,8441.000000,8086.000000,8093.000000,8.861000e+03
mean,52.163977,5.176015,1.962167,108.532717,0.977416,112.836456,8.529373e+08
std,18.919142,23.918947,0.877115,37.362528,0.197744,37.264390,7.607096e+06
min,1.000000,0.005000,0.050000,2.000000,0.170000,1.400000,8.408010e+08
25%,37.000000,0.447500,1.500000,87.000000,0.860000,93.000000,8.504031e+08
50%,55.000000,1.400000,1.900000,104.000000,0.960000,109.000000,8.510030e+08
75%,68.000000,2.700000,2.300000,126.000000,1.060000,128.000000,8.607110e+08
max,97.000000,530.000000,18.000000,600.000000,2.330000,642.000000,8.701190e+08


## Mapping of target column with corresponding classes

In [309]:
# Mapping dictionary based on provided mapping
mapping = {
    'A': 'Hyperthyroid', 'B': 'Hyperthyroid', 'C': 'Hyperthyroid', 'D': 'Hyperthyroid', 'AK': 'Hyperthyroid',
    'E': 'Hypothyroid', 'F': 'Hypothyroid', 'G': 'Hypothyroid', 'H': 'Hypothyroid', 'GK': 'Hypothyroid',
    'GI': 'Hypothyroid', 'FK': 'Hypothyroid', 'GKJ': 'Hypothyroid',
    'I': 'Binding Protein', 'J': 'Binding Protein', 'C | I': 'Binding Protein',
    'K': 'General Health', 'KJ': 'General Health', 'H | K': 'General Health',
    'M': 'Replacement Therapy', 'L': 'Replacement Therapy', 'N': 'Replacement Therapy',
    'MK': 'Replacement Therapy', 'MI': 'Replacement Therapy', 'LJ': 'Replacement Therapy',
    'O': 'Antithyroid Treatment', 'P': 'Antithyroid Treatment', 'Q': 'Antithyroid Treatment', 'OI': 'Antithyroid Treatment',
    'R': 'Miscellaneous', 'S': 'Miscellaneous', 'T': 'Miscellaneous', 'D | R': 'Miscellaneous',
    '-': 'No Condition'
}

# Create a new column based on the mapping
df['class'] = df['target'].map(mapping)

<ipython-input-309-3adfd7474aa4>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['class'] = df['target'].map(mapping)


In [310]:
# Creating a new Patient_ID column with unique IDs
df['Patient_ID']=df.index+1

<ipython-input-310-854083281711>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Patient_ID']=df.index+1


In [311]:
df

,age,sex,on_thyroxine,query_on_thyroxine,on_antithyroid_meds,sick,pregnant,thyroid_surgery,I131_treatment,query_hypothyroid,...,TSH,T3,TT4,T4U,FTI,referral_source,target,patient_id,class,Patient_ID
0,29,F,f,f,f,f,f,f,f,t,...,0.3,NaN,NaN,NaN,NaN,other,-,840801013,No Condition,1
1,29,F,f,f,f,f,f,f,f,f,...,1.6,1.9,128.0,NaN,NaN,other,-,840801014,No Condition,2
2,41,F,f,f,f,f,f,f,f,f,...,NaN,NaN,NaN,NaN,NaN,other,-,840801042,No Condition,3
3,36,F,f,f,f,f,f,f,f,f,...,NaN,NaN,NaN,NaN,NaN,other,-,840803046,No Condition,4
4,32,F,f,f,f,f,f,f,f,f,...,NaN,NaN,NaN,NaN,NaN,other,S,840803047,Miscellaneous,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8856,56,M,f,f,f,f,f,f,f,f,...,NaN,NaN,64.0,0.83,77.0,SVI,-,870119022,No Condition,8857
8857,22,M,f,f,f,f,f,f,f,f,...,NaN,NaN,91.0,0.92,99.0,SVI,-,870119023,No Condition,8858
8858,69,M,f,f,f,f,f,f,f,f,...,NaN,NaN,113.0,1.27,89.0,SVI,I,870119025,Binding Protein,8859
8859,47,F,f,f,f,f,f,f,f,f,...,NaN,NaN,75.0,0.85,88.0,other,-,870119027,No Condition,8860


In [312]:
# Dropping columns not requried
df.drop(columns=['target','patient_id'], axis=1, inplace=True)

<ipython-input-312-8fc691abfedd>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(columns=['target','patient_id'], axis=1, inplace=True)


In [313]:
df

,age,sex,on_thyroxine,query_on_thyroxine,on_antithyroid_meds,sick,pregnant,thyroid_surgery,I131_treatment,query_hypothyroid,...,hypopituitary,psych,TSH,T3,TT4,T4U,FTI,referral_source,class,Patient_ID
0,29,F,f,f,f,f,f,f,f,t,...,f,f,0.3,NaN,NaN,NaN,NaN,other,No Condition,1
1,29,F,f,f,f,f,f,f,f,f,...,f,f,1.6,1.9,128.0,NaN,NaN,other,No Condition,2
2,41,F,f,f,f,f,f,f,f,f,...,f,f,NaN,NaN,NaN,NaN,NaN,other,No Condition,3
3,36,F,f,f,f,f,f,f,f,f,...,f,f,NaN,NaN,NaN,NaN,NaN,other,No Condition,4
4,32,F,f,f,f,f,f,f,f,f,...,f,f,NaN,NaN,NaN,NaN,NaN,other,Miscellaneous,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8856,56,M,f,f,f,f,f,f,f,f,...,f,f,NaN,NaN,64.0,0.83,77.0,SVI,No Condition,8857
8857,22,M,f,f,f,f,f,f,f,f,...,f,f,NaN,NaN,91.0,0.92,99.0,SVI,No Condition,8858
8858,69,M,f,f,f,f,f,f,f,f,...,f,f,NaN,NaN,113.0,1.27,89.0,SVI,Binding Protein,8859
8859,47,F,f,f,f,f,f,f,f,f,...,f,f,NaN,NaN,75.0,0.85,88.0,other,No Condition,8860


In [314]:
df.head()

,age,sex,on_thyroxine,query_on_thyroxine,on_antithyroid_meds,sick,pregnant,thyroid_surgery,I131_treatment,query_hypothyroid,...,hypopituitary,psych,TSH,T3,TT4,T4U,FTI,referral_source,class,Patient_ID
0,29,F,f,f,f,f,f,f,f,t,...,f,f,0.3,NaN,NaN,NaN,NaN,other,No Condition,1
1,29,F,f,f,f,f,f,f,f,f,...,f,f,1.6,1.9,128.0,NaN,NaN,other,No Condition,2
2,41,F,f,f,f,f,f,f,f,f,...,f,f,NaN,NaN,NaN,NaN,NaN,other,No Condition,3
3,36,F,f,f,f,f,f,f,f,f,...,f,f,NaN,NaN,NaN,NaN,NaN,other,No Condition,4
4,32,F,f,f,f,f,f,f,f,f,...,f,f,NaN,NaN,NaN,NaN,NaN,other,Miscellaneous,5
